In [1]:
import fitz
from glob import *
import os
import pdfplumber
import time
from tqdm.notebook import tqdm

In [2]:
PATH_NOTES = r"G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES"
chemin_notes = sorted(glob(PATH_NOTES + r"\*.pdf", ), key=lambda x: int(x.split("\\")[-1].split("_")[0]))
# Pour le moment
chemin_notes.remove('G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\30_airbnb_JURIDIQUE_2024.pdf')
nom_notes = [chemin.split('\\')[-1].split('.')[0] for chemin in chemin_notes]
numero_notes = [int(chemin.split('\\')[-1].split("_")[0]) for chemin in chemin_notes]
chemin_notes[-2:], nom_notes[-2:], numero_notes[-2:]

(['G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\29_Loyers_2024.pdf',
  'G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\30_airbnb_DIAG_2024.pdf'],
 ['29_Loyers_2024', '30_airbnb_DIAG_2024'],
 [29, 30])

In [3]:
def remove_occurences_3(text):
    text_copy = text
    li = []
    # Trouve les suites de charactères à 3 occurences et rajoute les indices dans li (pas le premier)
    for i, _ in enumerate(text_copy[:-2]):
        if text_copy[i:i+3] == text_copy[i]*3:
            j=1
            while text_copy[i+j] == text_copy[i]:
                j += 1
                li.append(i+j-1)

    # Supprime les indices qui sont au milieu d'une suite
    lb = set(li)
    for el in set(li):
        if el + 1 in set(li) and el - 1 in set(li):
            lb.remove(el)

    # Supprime les suites
    lc = sorted(list(lb))
    for i in range(0, len(lc), 2):
        text_copy = text_copy.replace(text[lc[i]:lc[i+1]+1], '')
        # print(text[lc[i]:lc[i+1]+1], 'remplacée par ""')
    
    return text_copy

In [4]:
# PATH_NOTES / preprocessing / "nom_note" / texte
prepro_dir = 'preprocessing'
os.makedirs(os.path.join(PATH_NOTES, prepro_dir), exist_ok=True)
texte_dir = 'texte'

for chemin, nom, numero in tqdm(zip(chemin_notes, nom_notes, numero_notes), desc="Processing Notes"):    
    pdf = pdfplumber.open(chemin)
    note_dir = f"{nom.split('.')[0]}"
    page_dir = os.path.join(PATH_NOTES, prepro_dir, note_dir, texte_dir)
    os.makedirs(page_dir, exist_ok=True)

    for idx, page in tqdm(enumerate(pdf.pages), desc="Processing Pages", leave=False):
        text = page.extract_text()
        text_clean = remove_occurences_3(text)
        with open(f"{page_dir}/page_{idx+1}.txt", "w", encoding='utf-8') as f:
            f.write(text_clean)



Processing Notes: 0it [00:00, ?it/s]

Processing Pages: 0it [00:00, ?it/s]

Processing Pages: 0it [00:00, ?it/s]

Processing Pages: 0it [00:00, ?it/s]

Processing Pages: 0it [00:00, ?it/s]

...

Processing Pages: 0it [00:00, ?it/s]

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing Pages: 0it [00:00, ?it/s]

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing Pages: 0it [00:00, ?it/s]

In [6]:
os.path.join(PATH_NOTES, prepro_dir, note_dir, texte_dir)
for note in glob(os.path.join(PATH_NOTES, prepro_dir, '*')):
    print(note)
    print(len(glob(os.path.join(note, texte_dir, '*.txt'))))

G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\08_Loyers_2014
27
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\04_Loyers_2013
12
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\30_airbnb_DIAG_2024
21
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\05_Accession_2014
16
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\29_Loyers_2024
23
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\preprocessing\24_Accession_2021
11
G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES\prepro